In [1]:
import pandas as pd
import random
import os
import numpy as np
import datetime
from sklearn.utils import shuffle
import neptune

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

from torch.nn.utils.rnn import pack_padded_sequence,pad_sequence, pad_packed_sequence

import warnings
warnings.filterwarnings(action='ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

In [2]:
from dataset import *
from utils import *
from models import *
from attention_modules import *
from train import *
from test import *

In [3]:
import argparse

In [18]:
from config import get_Config

usage: ipykernel_launcher.py [-h] [--model_path MODEL_PATH] [--path PATH]
                             [--date1 DATE1] [--date2 DATE2]
                             [--new_sequence NEW_SEQUENCE]
                             [--input_size INPUT_SIZE]
                             [--output_size OUTPUT_SIZE]
                             [--hidden_size HIDDEN_SIZE]
                             [--sequence_length SEQUENCE_LENGTH]
                             [--num_layers NUM_LAYERS] [--dropout DROPOUT]
                             [--out_type OUT_TYPE] [--epoch EPOCH]
                             [--batch_size BATCH_SIZE] [--lr LR]
                             [--att_type ATT_TYPE] [--time_param TIME_PARAM]
                             [--feature_param FEATURE_PARAM] [--device DEVICE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\windd\AppData\Roaming\jupyter\runtime\kernel-7ca9c40a-790b-403a-98cc-f0461a07d030.json


SystemExit: 2

In [4]:
class Args:
    path = './data/ship_paper_50_2.csv'
    model_path = 'weights/'
    date1 = '2019-05'
    date2 = '2019-08'
    new_sequence = False
    
    model_setting = {'input_size': 16,
                     'output_size': 1,
                     'hidden_size': 4,
                     'sequence_length': 50,
                     'num_layers': 2,
                     'dropout': 0.1}
    
    model_setting_ens = {'input_size': 16,
                     'output_size': 1,
                     'hidden_size': 4,
                     'sequence_length': 50,
                     'num_layers': 2,
                     'dropout': 0.1,    
                     'out_type': 'representation'}    
    epoch = 75
    batch_size = 200
    lr = 0.001
    att_type = 'ENS' #SA FA ENS
    
    time_param = 800
    feature_param = int(0.9*50) # sequence length
    
    device = 'cuda:0'
    
args = Args()    

In [18]:
class Args:
    path = './data/ship_paper_50_2.csv'
    model_path = 'weights/'
    date1 = '2019-05'
    date2 = '2019-08'
    new_sequence = False
    
    input_size= 16
    output_size= 1
    hidden_size= 4
    sequence_length= 50
    num_layers= 2
    dropout= 0.1
    out_type= 'representation'    
    epoch = 75
    batch_size = 200
    lr = 0.001
    att_type = 'TA' #SA FA ENS
    
    time_param = 800
    feature_param = int(0.9*50) # sequence length
    
    device = 'cuda:0'
    
args = Args()    

In [6]:
total_data, train_mean, train_std, test_mean, test_std, train_utc_mean, \
train_utc_std, test_utc_mean, test_utc_std  = get_data(args.path, args.date1, args.date2, 
                                                       args.sequence_length,args.new_sequence)

train = ship_data(total_data, train_mean, train_std, train_utc_mean, train_utc_std, '2013', args.date1)
val = ship_data(total_data, train_mean, train_std,  train_utc_mean, train_utc_std, args.date1, args.date2)
test = ship_data(total_data, test_mean, test_std,  test_utc_mean, test_utc_std, args.date2, '2020')

train_loader = DataLoader(train, batch_size = args.batch_size,collate_fn = pad_collate,shuffle=False)
val_loader = DataLoader(val, batch_size = args.batch_size,collate_fn = pad_collate, shuffle = False)
test_loader = DataLoader(test, batch_size = args.batch_size,collate_fn = pad_collate, shuffle = False)

In [19]:
# log setting

trainer = Trainer(args)
tester = Tester(args)

PARAMS = {'epoch':args.epoch,
          'batch_size':args.batch_size,
          'lr':args.lr,
          'att_type':args.att_type,
          'device': args.device}
PARAMS.update(trainer.model_setting)
print(PARAMS)

if args.att_type == 'ENS':
    trainer.train_ens(train_loader, val_loader)
    tester.test_ens(test_loader)
else:
    trainer.train(train_loader, val_loader)
    tester.test(test_loader)

# experiment_name = 'Experiment'
# tag_name = ['Tag']
# neptune_load(experiment_name, tag_name, PARAMS)

{'epoch': 75, 'batch_size': 200, 'lr': 0.001, 'att_type': 'TA', 'device': 'cuda:0', 'input_size': 16, 'output_size': 1, 'hidden_size': 4, 'sequence_length': 50, 'num_layers': 2, 'dropout': 0.1}
TA model training
epoch: 75/1  | trn loss: 3184.7359 | val loss: 2822.2146


KeyboardInterrupt: 